# Example of computing vibrational energy levels of a triatomic molecule using the Taylor series expansion approximation for the kinetic and potential energy operators

Compute vibrational states of $\text{H}_2\text{S}$ molecule using the potential energy surface from
[A. A. A. Azzam, J. Tennyson, S. N. Yurchenko, O. V. Naumenko, "ExoMol molecular line lists - XVI. The rotation–vibration spectrum of hot H2S", MNRAS 460, 4063–4074 (2016)](https://doi.org/10.1093/mnras/stw1133). See Python implementation in [h2s_AYT2.py](../vibrojet/potentials/h2s_AYT2.py).


In [1]:
import itertools
import os

import jax
import jax.numpy as jnp
import numpy as np
from scipy import optimize

from vibrojet.basis_utils import ContrBasis, HermiteBasis
from vibrojet.keo import Gmat, pseudo, com
from vibrojet.potentials import h2s_AYT2
from vibrojet.taylor import deriv_list

jax.config.update("jax_enable_x64", True)

Compute equilibrium coordinates, around which the Taylor series expansions will be built

In [2]:
vmin = optimize.minimize(h2s_AYT2.poten, [1, 1, 90 * np.pi / 180])
r0 = vmin.x
v0 = vmin.fun
print("Equilibrium coordinates:", r0)
print("Min of the potential:", v0)

Equilibrium coordinates: [1.3358387  1.3358387  1.61042427]
Min of the potential: -0.0007846164047977397


Define mapping function from internal valence coordinates, $r_1$, $r_2$, and $\alpha$, to Cartesian coordinates

In [3]:
masses = [31.97207070, 1.00782505, 1.00782505]  # masses of S, H, H

ncoo = len(r0)


@com(masses)
def internal_to_cartesian(coords):
    r1, r2, alpha = coords
    return jnp.array(
        [
            [0.0, 0.0, 0.0],
            [r1 * jnp.cos(alpha / 2), 0.0, r1 * jnp.sin(alpha / 2)],
            [r2 * jnp.cos(alpha / 2), 0.0, -r2 * jnp.sin(alpha / 2)],
        ]
    )

Compute Taylor series expansion of the $G$-matrix and potential

$$
G_{\lambda,\mu}=\sum_{\mathbf{t}} g_{\mathbf{t},\lambda,\mu} (r_1-r_1^\text{(eq)})^{t_1}(r_2-r_2^\text{(eq)})^{t_2}(\alpha-\alpha^\text{(eq)})^{t_3},
$$

$$
V=\sum_{\mathbf{t}} f_{\mathbf{t}} (r_1-r_1^\text{(eq)})^{t_1}(r_2-r_2^\text{(eq)})^{t_2}(\alpha-\alpha^\text{(eq)})^{t_3},
$$

 where the derivative multi-index $\mathbf{t}$ is stored in `gmat_terms` and `poten_terms`, and the expansion coefficients $g_{\mathbf{t},\lambda,\mu}$ and $f_{\mathbf{t}}$ in `gmat_coefs[:len(gmat_terms),:3N, :3N]` and `poten_coefs[:len(poten_terms)]` respectively, where $N$ is the number of atoms


In [5]:
gmat_max_order = 8  # max total expansion order

gmat_terms = np.array(
    [
        elem
        for elem in itertools.product(
            *[range(0, gmat_max_order + 1) for _ in range(len(r0))]
        )
        if sum(elem) <= gmat_max_order
    ]
)
pseudo_terms = gmat_terms
print("max expansion order for G-matrix:", gmat_max_order)
print("number of expansion terms in G-matrix:", len(gmat_terms))

# expansion of G-matrix

gmat_file = f"_h2s_gmat_coefs_{gmat_max_order}.npy"
if os.path.exists(gmat_file):
    print(
        f"load G-matrix expansion coefs from file {gmat_file} (delete file to recompute coefficients)"
    )
    gmat_coefs = np.load(gmat_file)
else:
    gmat_coefs = deriv_list(
        lambda x: Gmat(x, masses, internal_to_cartesian), gmat_terms, r0, if_taylor=True
    )
    np.save(gmat_file, gmat_coefs)

# expansion of pseudopotential

pseudo_file = f"_h2s_pseudo_coefs_{gmat_max_order}.npy"
if os.path.exists(pseudo_file):
    print(
        f"load pseudopotential expansion coefs from file {pseudo_file} (delete file to recompute coefficients)"
    )
    pseudo_coefs = np.load(pseudo_file)
else:
    pseudo_coefs = deriv_list(
        lambda x: pseudo(x, masses, internal_to_cartesian),
        gmat_terms,
        r0,
        if_taylor=True,
    )
    np.save(pseudo_file, pseudo_coefs)

max expansion order for G-matrix: 8
number of expansion terms in G-matrix: 165
load G-matrix expansion coefs from file _h2s_gmat_coefs_8.npy (delete file to recompute coefficients)
load pseudopotential expansion coefs from file _h2s_pseudo_coefs_8.npy (delete file to recompute coefficients)


In [6]:
poten_max_order = 8

poten_terms = np.array(
    [
        elem
        for elem in itertools.product(
            *[range(0, poten_max_order + 1) for _ in range(len(r0))]
        )
        if sum(elem) <= poten_max_order
    ]
)
print("max expansion order for PES:", poten_max_order)
print("number of expansion terms in PES:", len(poten_terms))

poten_file = f"_h2s_poten_coefs_{poten_max_order}.npy"
if os.path.exists(poten_file):
    print(
        f"load potential expansion coefs from file {poten_file} (delete file to recompute coefficients)"
    )
    poten_coefs = np.load(poten_file)
else:
    poten_coefs = deriv_list(h2s_AYT2.poten, poten_terms, r0, if_taylor=True)
    np.save(poten_file, poten_coefs)

max expansion order for PES: 8
number of expansion terms in PES: 165
load potential expansion coefs from file _h2s_poten_coefs_8.npy (delete file to recompute coefficients)


Define mapping between the coordinate $x\in (-\infty, \infty)$ of Hermite basis functions and the internal valence coordinates as $r_1=a_1 x_1 + b_1$, $r_2=a_2x_2+b_2$, $\alpha=a_3x_3+b_3$. The parameters $a_1,b_1,...,b_3$ are determined by mapping the vibrational Hamiltonian in valence coordinates onto the harmonic-oscillator Hamiltonian.

In [7]:
mask = gmat_terms != 0  # Mask for derivatives

# Vibrational G-matrix elements at equilibrium geometry
ind0 = np.where(mask.sum(axis=1) == 0)[0][0]
mu = np.diag(gmat_coefs[ind0])[:ncoo]

# Second-order derivative of potential at equilibrium
mask = poten_terms != 0  # Mask for derivatives
ind2 = np.array(
    [   np.where((mask.sum(axis=1) == 1) & (poten_terms[:, icoo] == 2))[0][0]
        for icoo in range(ncoo)]
)

freq = poten_coefs[ind2] * 2

# Linear mapping parameters
lin_a = jnp.sqrt(jnp.sqrt(mu / freq))
lin_b = r0

print("x->r linear mapping parameters 'a':", lin_a)
print("x->r linear mapping parameters 'b':", lin_b)

# Linear mapping function
x_to_r_map = lambda x, icoo: lin_a[icoo] * x + lin_b[icoo]

x->r linear mapping parameters 'a': [0.11245619 0.11245619 0.17845517]
x->r linear mapping parameters 'b': [1.3358387  1.3358387  1.61042427]


Generate primitive Hermite basis sets for each coordinate.

In [8]:
nbas = [30, 30, 24]
npoints = [80] * ncoo

bas_r1, bas_r2, bas_alpha = [
    HermiteBasis(
        icoo,
        nbas[icoo],
        npoints[icoo],
        lambda x: x_to_r_map(x, icoo),
        lambda r: r - r0[icoo],
        lambda r: r - r0[icoo],
        gmat_terms[:, icoo],
        poten_terms[:, icoo],
        pseudo_terms[:, icoo],
    )
    for icoo in range(ncoo)
]

Compute 1D contracted basis sets, by solving 1D reduced-mode Schrödinger equations for each coordinate.

In [9]:
bas_r1, bas_r2, bas_alpha = [
    ContrBasis(
        (icoo,),
        [bas_r1, bas_r2, bas_alpha],
        lambda _: True,
        gmat_terms,
        gmat_coefs,
        poten_terms,
        poten_coefs,
        emax=40000,
    )
    for icoo in range(ncoo)
]

e = bas_r1.enr
print("Solutions for r1 mode:\n", f"zpe = {e[0]}\n", e - e[0])
e = bas_r2.enr
print("\nSolutions for r2 mode:\n", f"zpe = {e[0]}\n", e - e[0])
e = bas_alpha.enr
print("\nSolutions for alpha mode:\n", f"zpe = {e[0]}\n", e - e[0])

Solutions for r1 mode:
 zpe = 2361.7950570907633
 [    0.          2627.83310095  5165.68583648  7629.77901979
 10056.05642783 12496.83825129 15001.60164644 17601.69818093
 20311.16466355 23134.07657558 26069.98236833 29116.65213762
 32271.47762591 35532.58893808 38899.21252554]

Solutions for r2 mode:
 zpe = 2361.795057137522
 [    0.          2627.83310095  5165.68583648  7629.77901976
 10056.05642772 12496.83825106 15001.60164602 17601.69818028
 20311.16466265 23134.07657442 26069.98236687 29116.65213587
 32271.47762385 35532.5889357  38899.21252283]

Solutions for alpha mode:
 zpe = 2026.5007139178242
 [    0.          1214.1828581   2422.52098472  3624.57606553
  4819.78084361  6007.22739247  7185.15140905  7351.13931048
  8350.72917815  9496.80946149 10608.78971238 11656.6267001
 12623.30073129 13598.36265685 14676.14362905 15845.09195459
 17086.46640086 18383.87068341 19733.52931523 21161.16514744
 22547.85661979 24182.30837895 25650.8574639  27516.3243721 ]


Compute 2D contracted basis sets for the stretching coordinates $r_1$ and $r_2$ by solving 2D reduced-mode Schrödinger equation, where basis functions for the bending  $\alpha$ coordinate are fixed to the ground (zero-order function) state.

In [10]:
bas_r1_r2 = ContrBasis(
    (0, 1),
    (bas_r1, bas_r2, bas_alpha),
    lambda _: True,
    gmat_terms,
    gmat_coefs,
    poten_terms,
    poten_coefs,
    emax=40000,
)

e = bas_r1_r2.enr
print("Solutions for r1+r2 modes:\n", f"zpe = {e[0]}\n", e - e[0])

Solutions for r1+r2 modes:
 zpe = 3007.6990842051514
 [    0.          2619.90705622  2634.82764618  5162.02004741
  5164.09720811  5256.79159799  7624.76498937  7624.98085051
  7777.2518065   7808.28533577 10043.99256128 10044.16337104
 10250.03734173 10254.24401492 10341.27551785 12465.89841443
 12465.94175046 12675.57103761 12680.93938133 12784.75945293
 12831.34790893 14922.10135492 14922.27900747 15113.47410355
 15120.61312068 15235.94472002 15244.33375269 15316.87690823
 17411.82523568 17413.00626676 17589.93727758 17604.01647165
 17706.92091671 17711.14351955 17781.88520798 17826.13459693
 19926.42178329 19931.35067989 20078.91950456 20104.37418121
 20207.19846206 20212.1325486  20283.9414635  20445.75480963
 20455.28155645 22468.17998969 22481.6613593  22579.50020085
 22611.33678443 22715.3775176  22764.18869705 22885.55595227
 22926.84319214 23244.61820117 23246.1718497  25034.64515157
 25051.05211157 25132.41279492 25139.35232896 25296.10685239
 25380.76832718 25394.23143007 

Compute full-dimensional solutions using contracted basis sets for all coordinates.
The size of the total basis set is controlled by the scaled sum of the stretching and bending excitation numbers in `select_quanta`

In [11]:
p_max = 10


def select_quanta(ind):
    ind_str, ind_bnd = ind
    cond = int(np.ceil(ind_str / 2)) * 2 + ind_bnd <= p_max
    return cond


bas_r1_r2_alpha = ContrBasis(
    (0, 1),
    (bas_r1_r2, bas_alpha),
    select_quanta,
    gmat_terms,
    gmat_coefs,
    poten_terms,
    poten_coefs,
    store_me=False,
)

e = bas_r1_r2_alpha.enr
print(len(e))
print("Solutions for r1+r2+alpha modes:\n", f"zpe = {e[0]}\n", e - e[0])

61
Solutions for r1+r2+alpha modes:
 zpe = 3300.83022101828
 [    0.          1182.69016146  2354.10279399  2614.9705612
  2629.19007474  3513.82469533  3780.07418898  3790.2972795
  4661.41343473  4933.86307634  4940.45529102  5151.23368443
  5153.67794154  5245.3207521   5796.70193169  5896.8296015
  6076.35931497  6079.3534815   6295.48610317  6298.43974798
  6388.02678711  6917.52530151  7206.10678282  7214.98781926
  7433.50287485  7444.76751967  7533.36670098  7613.67898851
  7614.24518363  7768.54929475  7798.43980148  8049.02697202
  8319.75897526  8370.42791152  8558.92358929  8612.98979777
  8717.06681379  8738.6279759   8758.27648267  8761.16398798
  8920.74050105  9280.743632    9447.20382985  9522.25558291
  9704.38260578  9765.05488968  9886.02236304  9909.55007997
  9924.11230067 10001.7368067  10037.36172024 10086.96779526
 10376.56141636 10664.64673176 10793.67283321 10920.46668403
 10991.88208941 11072.1918288  12052.34260723 12124.58426125
 12240.2500752 ]
